In [2]:
import os
import librosa
import numpy as np

# 定义文件夹路径
folders = {
    "WhiteBeaked": r"D:\Dataset\Data_Watkins\Best_WhiteBeaked_sounds",
    "AtlanticSpotted": r"D:\Dataset\Data_Watkins\Best_AtlanticSpotted_sounds",
    "WhiteSided": r"D:\Dataset\Data_Watkins\Best_WhiteSided_sounds"
}

def compute_snr_lhr_adaptive(signal, sample_rate):
    """
    计算 SNR（信噪比）和 LHR（低频/高频比）
    """
    # 计算傅里叶变换
    fft_signal = np.fft.fft(signal)
    psd_signal = np.abs(fft_signal) ** 2  # 计算功率谱密度
    freqs = np.fft.fftfreq(len(signal), d=1/sample_rate)
    freqs = freqs[:len(freqs) // 2]
    psd_signal = psd_signal[:len(psd_signal) // 2]

    # --- 计算能量中心 ---
    spectral_centroid = np.sum(freqs * psd_signal) / (np.sum(psd_signal) + 1e-8)

    # --- 计算 SNR ---
    geometric_mean = np.exp(np.mean(np.log(psd_signal + 1e-8)))  # 避免 log(0)
    arithmetic_mean = np.mean(psd_signal)
    sfm = geometric_mean / (arithmetic_mean + 1e-8)
    threshold = np.percentile(psd_signal, 90 * (1 - sfm))
    signal_power = np.sum(psd_signal[psd_signal > threshold])
    noise_power = np.sum(psd_signal[psd_signal <= threshold])
    snr = 10 * np.log10(signal_power / (noise_power + 1e-8))

    # --- 计算 LHR ---
    low_freq_power = np.sum(psd_signal[freqs < spectral_centroid])
    high_freq_power = np.sum(psd_signal[freqs >= spectral_centroid])
    lhr = low_freq_power / (high_freq_power + 1e-8)

    return snr, lhr

# 遍历每个文件夹
stats = {}

for label, folder in folders.items():
    print(f"\n📂 处理文件夹: {folder} ({label})")

    # 初始化统计计数
    lhr_gt_1 = 0  # LHR > 1 的数量
    lhr_lt_1 = 0  # LHR < 1 的数量
    snr_bins = { "0-10": 0, "10-20": 0, "20-30": 0, "30+": 0 }  # SNR 分布

    # 遍历文件夹中的音频文件
    for filename in os.listdir(folder):
        if filename.endswith(".wav"):  # 只处理 WAV 文件
            file_path = os.path.join(folder, filename)

            # 读取音频文件
            signal, sr = librosa.load(file_path, sr=None)

            # 计算 SNR 和 LHR
            snr, lhr = compute_snr_lhr_adaptive(signal, sr)

            # 更新 LHR 计数
            if lhr > 1:
                lhr_gt_1 += 1
            else:
                lhr_lt_1 += 1

            # 更新 SNR 计数
            if 0 <= snr < 10:
                snr_bins["0-10"] += 1
            elif 10 <= snr < 20:
                snr_bins["10-20"] += 1
            elif 20 <= snr < 30:
                snr_bins["20-30"] += 1
            else:
                snr_bins["30+"] += 1

    # 存储统计结果
    stats[label] = {
        "LHR > 1": lhr_gt_1,
        "LHR < 1": lhr_lt_1,
        "SNR 分布": snr_bins
    }

# 打印统计结果
print("\n📊 数据统计结果：")
for label, data in stats.items():
    print(f"\n🔹 {label}:")
    print(f"   LHR > 1: {data['LHR > 1']}  |  LHR < 1: {data['LHR < 1']}")
    print(f"   SNR 0-10: {data['SNR 分布']['0-10']}  |  SNR 10-20: {data['SNR 分布']['10-20']}")
    print(f"   SNR 20-30: {data['SNR 分布']['20-30']}  |  SNR 30+: {data['SNR 分布']['30+']}")



📂 处理文件夹: D:\Dataset\Data_Watkins\Best_WhiteBeaked_sounds (WhiteBeaked)

📂 处理文件夹: D:\Dataset\Data_Watkins\Best_AtlanticSpotted_sounds (AtlanticSpotted)

📂 处理文件夹: D:\Dataset\Data_Watkins\Best_WhiteSided_sounds (WhiteSided)

📊 数据统计结果：

🔹 WhiteBeaked:
   LHR > 1: 49  |  LHR < 1: 8
   SNR 0-10: 10  |  SNR 10-20: 34
   SNR 20-30: 13  |  SNR 30+: 0

🔹 AtlanticSpotted:
   LHR > 1: 16  |  LHR < 1: 42
   SNR 0-10: 33  |  SNR 10-20: 23
   SNR 20-30: 2  |  SNR 30+: 0

🔹 WhiteSided:
   LHR > 1: 36  |  LHR < 1: 19
   SNR 0-10: 24  |  SNR 10-20: 30
   SNR 20-30: 1  |  SNR 30+: 0


In [ ]:
import os
import numpy as np
import librosa

# 定义文件夹路径
folder_1 = r"D:\Dataset\Data_Watkins\Best_WhiteBeaked_sounds"
folder_2 = r"D:\Dataset\Data_Watkins\Best_AtlanticSpotted_sounds"
folder_3 = r"D:\Dataset\Data_Watkins\Best_WhiteSided_sounds"

# 目标采样点数
target_length = 50000

# 处理音频文件的函数
def process_audio_files(folder):
    processed_audios = []
    audio_ids = []  # 存储文件 ID

    for file in os.listdir(folder):
        if file.endswith(".wav"):
            file_path = os.path.join(folder, file)

            # 读取音频
            audio, sr = librosa.load(file_path, sr=None)  # 保持原始采样率
            total_samples = len(audio)

            # 记录文件 ID（不包含扩展名）
            file_id = os.path.splitext(file)[0]

            # 如果音频长度不足 target_length，则前面补零
            if total_samples < target_length:
                audio = np.pad(audio, (target_length - total_samples, 0), mode='constant')
                processed_audios.append(audio)
                audio_ids.append(file_id)  # 记录 ID
            else:
                # 计算应切分的段数
                num_segments = max(1, round(total_samples / target_length))
                segment_length = total_samples // num_segments

                for i in range(num_segments):
                    start = i * segment_length
                    end = min(start + target_length, total_samples)  # 确保不越界

                    segment = audio[start:end]  # 默认切片
                    if len(segment) < target_length:
                        segment = np.pad(segment, (0, target_length - len(segment)), mode='constant')  # 不足时填充0

                    processed_audios.append(segment)
                    audio_ids.append(f"{file_id}_seg{i}")  # 记录分割后的 ID

    return np.array(processed_audios), np.array(audio_ids)

# 处理三个文件夹
data_whitebeaked, ids_whitebeaked = process_audio_files(folder_1)
data_atlanticspotted, ids_atlanticspotted = process_audio_files(folder_2)
data_whitesided, ids_whitesided = process_audio_files(folder_3)

# 对每个样本归一化到 [-1, 1]
def normalize_audio(data):
    return np.array([sample / np.max(np.abs(sample)) if np.max(np.abs(sample)) > 0 else sample for sample in data])

data_whitebeaked = normalize_audio(data_whitebeaked)
data_atlanticspotted = normalize_audio(data_atlanticspotted)
data_whitesided = normalize_audio(data_whitesided)

# 保存音频数据
np.save("whitebeaked_sounds.npy", data_whitebeaked)
np.save("atlanticspotted_sounds.npy", data_atlanticspotted)
np.save("whitesided_sounds.npy", data_whitesided)

# 保存音频 ID（可选择保存为 .npy 或 .txt）
np.save("whitebeaked_ids.npy", ids_whitebeaked)
np.save("atlanticspotted_ids.npy", ids_atlanticspotted)
np.save("whitesided_ids.npy", ids_whitesided)

# 也可以保存为可读的 .txt 文件
np.savetxt("whitebeaked_ids.txt", ids_whitebeaked, fmt="%s")
np.savetxt("atlanticspotted_ids.txt", ids_atlanticspotted, fmt="%s")
np.savetxt("whitesided_ids.txt", ids_whitesided, fmt="%s")

# 输出数组大小
print(f"WhiteBeaked sounds shape: {data_whitebeaked.shape}, IDs: {ids_whitebeaked.shape}")
print(f"AtlanticSpotted sounds shape: {data_atlanticspotted.shape}, IDs: {ids_atlanticspotted.shape}")
print(f"WhiteSided sounds shape: {data_whitesided.shape}, IDs: {ids_whitesided.shape}")

print("处理完成，数据及音频 ID 已保存。")
